In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


from lightgbm import LGBMClassifier
import gc
import os
import re

%aimport utils

In [ ]:
 train = utils.read_csv('./input/application_train.csv')
 test = utils.read_csv('./input/application_test.csv')
 prev = utils.read_csv('./input/previous_application.csv')
 buro = utils.read_csv('./input/bureau.csv')

Memory usage of dataframe is 286.23 MB


In [4]:
categorical_feats = [
    f for f in train.columns if train[f].dtype == 'object'
]

In [5]:
for f in categorical_feats:
    train[f], indexer = pd.factorize(train[f])
    test[f] = indexer.get_indexer(test[f])

In [6]:
gc.enable()

In [7]:
y = train['TARGET']
del train['TARGET']

In [8]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

In [9]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [10]:
buro_cat_features = [
    f_ for f_ in buro.columns if buro[f_].dtype == 'object'
]
for f_ in buro_cat_features:
    buro[f_], _ = pd.factorize(buro[f_])

avg_buro = buro.groupby('SK_ID_CURR').mean()
avg_buro['buro_count'] = buro[['SK_ID_BUREAU','SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']
del avg_buro['SK_ID_BUREAU']

In [11]:
train = train.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
train = train.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')

test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')


In [12]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in train.columns if f_ not in excluded_feats]

In [13]:
trn_x, val_x, trn_y, val_y = train_test_split(train[features], y,  test_size=0.2, random_state=42)

In [15]:
exp_id = utils.experiment_id()

clf = LGBMClassifier(
    n_estimators=20000,
    learning_rate=0.005,
    num_leaves=70,
    colsample_bytree=.8,
    subsample=.9,
    max_depth=7,
    reg_alpha=.1,
    reg_lambda=.1,
    min_split_gain=.01,
    min_child_weight=2,
    device="gpu"
)

clf.fit(trn_x, trn_y, 
        eval_set= [(trn_x, trn_y), (val_x, val_y)], 
        eval_metric='auc', verbose=250, early_stopping_rounds=150
       )

val_preds = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
test_preds = clf.predict_proba(test[features], num_iteration=clf.best_iteration_)[:, 1]

auc = roc_auc_score(val_y, val_preds)
print('AUC : %.6f' % auc)
gc.collect()

csv = f"experiments/{exp_id}-lgbm-{auc:.6f}.csv"

test['TARGET'] = test_preds
test[['SK_ID_CURR', 'TARGET']].to_csv(csv, index=False, float_format='%.8f')
print(f'Save to {csv}')

Training until validation scores don't improve for 150 rounds.
Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.701239	valid_1's auc: 0.696289
AUC : 0.696289
Save to experiments/003-lgbm-0.696289.csv
